# Dataset building

Pseudocode

```
Open raster
Open vector files
Reproject vector files to match raster's
For each chip in raster:
    Extract chip to directory
    Set label for chip with vector files
Write labels.txt
```

In [14]:
import rasterio
import fiona
import numpy as np
import os
import rtree
import pyproj
import csv
import shutil

from shapely.geometry import shape, box 
from shapely.ops import transform
from tqdm import tqdm_notebook
from rasterio.windows import Window
from skimage.io import imsave
from skimage.exposure import is_low_contrast
from functools import partial

In [6]:
DATA_DIR = os.path.join('..', 'data')

RASTER = os.path.join(DATA_DIR, 'montevideo', 'mosaic_jpg.tif')
AOI_VECTOR = os.path.join(DATA_DIR, 'montevideo', 'aoi.shp')
URBAN_VECTOR = os.path.join(DATA_DIR, 'montevideo', 'urban.shp')
RURAL_VECTOR = os.path.join(DATA_DIR, 'montevideo', 'rural.shp')
SCHOOL_VECTOR = os.path.join(DATA_DIR, 'montevideo', 'school.geojson')

DATASET_DIR = os.path.join(DATA_DIR, 'datasets', '1')
WIDTH = 300
HEIGHT = 300

CLASSES = dict(urban=URBAN_VECTOR, rural=RURAL_VECTOR, school=SCHOOL_VECTOR)

## Extract chips and labels from vector files

First define some useful functions for building R-Tree indexes and reprojecting shapes:

In [9]:
def create_index(shapes):
    """Create an R-Tree index from a set of shapes"""
    index = rtree.index.Index()
    for shape_id, shape in enumerate(shapes):
        index.insert(shape_id, shape.bounds)
    return index

def reproject_shape(shape, src_crs, dst_crs):
    """Reprojects a shape from some projection to another"""
    src_crs = dict(init=src_crs) if isinstance(src_crs, str) else src_crs
    dst_crs = dict(init=dst_crs) if isinstance(dst_crs, str) else dst_crs
    project = partial(
        pyproj.transform,
        pyproj.Proj(init=src_crs['init']),
        pyproj.Proj(init=dst_crs['init']))
    return transform(project, shape)

def get_raster_crs(raster_path):
    """Return CRS of +raster_path+"""
    with rasterio.open(raster_path) as dataset:
        return dataset.crs
    
def get_shapes(vector_path, target_crs=None):
    with fiona.open(vector_path) as dataset:
        shapes = (shape(f['geometry']) for f in dataset if f['geometry'])
        if dataset.crs != target_crs:
            shapes = (reproject_shape(s, dataset.crs, target_crs) for s in shapes)
        valid_shapes = [s for s in shapes if s.is_valid]
        return valid_shapes

def prepare_shapes_by_class(classes, vector_files, target_crs):
    """Load all shapes from each class vector file"""
    return {class_name: get_shapes(vector_file, target_crs)
            for class_name, vector_file in zip(classes, vector_files)}

def prepare_index_by_class(classes, shapes):
    """Build R-Tree index for all classes"""
    return {class_name: create_index(shapes)
            for class_name, shapes in zip(classes, shapes)}

In [10]:
# Load shapes for classes and build indexes for fast intersection
classes = tuple(CLASSES.keys())
raster_crs = get_raster_crs(RASTER)
        
print("Loading shapes...")
shapes_by_class = prepare_shapes_by_class(classes=classes,
                                          vector_files=tuple(CLASSES.values()),
                                          target_crs=raster_crs['init'])
print("Building indexes...")
index_by_class = prepare_index_by_class(classes=classes,
                                        shapes=tuple(shapes_by_class.values()))

print("Done")

Loading shapes...
Building indexes...
Done


In [13]:
def any_intersecting_shape(wbox, shapes, index):
    """Return true iif any shapes intersects with window, or is contained in window"""
    matching_shapes = [shapes[i] for i in index.intersection(wbox.bounds)]
    if matching_shapes:
        return any(shape.intersection(wbox) for shape in matching_shapes)
    else:
        return False

def get_window_labels(wbox, *, shapes_by_class, index_by_class):
    labels = {}
    any_label = False
    for class_name, shapes in shapes_by_class.items():
        index = index_by_class[class_name]
        labels[class_name] = any_intersecting_shape(wbox, shapes, index)
    return labels
        
def sliding_windows(size, step_size, width, height):
    """Slide a window of +size+ by moving it +step_size+ pixels"""
    w, h = size
    sw, sh = step_size
    for pos_i, i in enumerate(range(0, height - h + 1, sh)):
        for pos_j, j in enumerate(range(0, width - w + 1, sw)):
            yield Window(j, i, w, h), (pos_i, pos_j)

def write_image(img, path):
    rgb = np.dstack(img[:3, :, :])
    if is_low_contrast(rgb):
        return False
    os.makedirs(os.path.dirname(path), exist_ok=True)
    imsave(path, rgb)
    return True

def convert_bools(labels):
    return {k: (1 if v else 0) for k, v in labels.items()}

def build_dataset(raster, aoi_vector=None, *, class_vectors, shapes_by_class, index_by_class, width, height, output_dir):
    with rasterio.open(raster) as ds:
        print('Raster size: {}'.format((ds.width, ds.height)))
        raster_crs = ds.crs
        
        # Build window list
        size = (width, height)
        print("Building window list...")
        windows = list(sliding_windows(size, size, ds.width, ds.height))
        
        # TODO Filter windows with aoi_vector
        pass
    
        labels_path = os.path.join(output_dir, 'labels.csv')
        os.makedirs(os.path.dirname(labels_path), exist_ok=True)
        
        with open(labels_path, 'w', newline='') as csvfile:
            fieldnames = ['img'] + list(class_vectors.keys())
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
        
            for window, (i, j) in tqdm_notebook(windows):
                #print(window, (i, j))
                img = ds.read(window=window)

                img_filename = '{i}_{j}.jpg'.format(i=i, j=j)
                img_path = os.path.join(output_dir, 'all', img_filename)
                was_image_written = write_image(img, img_path)

                if was_image_written:
                    # Get window labels
                    wbox = box(*ds.window_bounds(window))
                    labels = get_window_labels(wbox, shapes_by_class=shapes_by_class, index_by_class=index_by_class)
                    labels = convert_bools(labels)

                    # Write labels to file
                    labels['img'] = img_filename
                    writer.writerow(labels)

In [14]:
build_dataset(RASTER,
              width=WIDTH,
              height=HEIGHT,
              aoi_vector=AOI_VECTOR,
              class_vectors=CLASSES,
              shapes_by_class=shapes_by_class,
              index_by_class=index_by_class,
              output_dir=DATASET_DIR)

Raster size: (109149, 76345)
Building window list...


## Train-test split

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
def parse_label_row(row):
    labels = list(row.values())[1:]
    labels = [int(label) for label in labels]
    return labels

def get_instances_and_labels_from_dataset(dataset_dir):
    labels_path = os.path.join(dataset_dir, 'labels.csv')
    with open(labels_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        rows = list(reader)
    X = np.array([row['img'] for row in rows])
    y = np.array([parse_label_row(row) for row in rows])
    return X, y

In [9]:
X, y = get_instances_and_labels_from_dataset(DATASET_DIR)

In [10]:
X, y

(array(['0_92.jpg', '0_93.jpg', '0_94.jpg', ..., '253_155.jpg',
        '253_156.jpg', '253_157.jpg'], dtype='<U11'), array([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]))

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

In [16]:
# Create symlinks
all_dir = os.path.abspath(os.path.join(DATASET_DIR, 'all'))
train_dir = os.path.abspath(os.path.join(DATASET_DIR, 'train'))
test_dir = os.path.abspath(os.path.join(DATASET_DIR, 'test'))

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for fname in X_train:
    shutil.copyfile(os.path.join(all_dir, fname), os.path.join(train_dir, fname))

for fname in X_test:
    shutil.copyfile(os.path.join(all_dir, fname), os.path.join(test_dir, fname))

## Class balancing

In [17]:
from sklearn.utils import class_weight

In [26]:
y_ints = [y.argmax() for y in y_train]
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_ints), y_ints)

In [27]:
class_weights

array([  0.42842266,   1.51206851, 221.69444444])

In [31]:
np.sum(y_train[:,2])

260

In [32]:
len(y_train)

39905